# Plotting Notebook

This notebook contains the moved 3D distal-finger trajectory plot.

## 1) Plotting context

In [ ]:
%matplotlib inline
from pathlib import Path
import sys

repo_root = Path.cwd()
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

print(f"Notebook root: {repo_root}")

## 2) Imports required for 3D plotting

In [ ]:
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as mcolors

## 3) Resolve local dependencies (`train_data`, `hand_names`, `signal_cols`)

In [ ]:
train_data = pd.read_csv('train.csv')

default_exclude = {'id', 'shot_id', 'angle', 'depth', 'left_right'}
signal_cols = [c for c in train_data.columns if c not in default_exclude]
hand_names = [c for c in signal_cols if ('finger' in c.lower() or 'wrist' in c.lower())]

def to_series_array(value):
    if isinstance(value, np.ndarray):
        return value.astype(float)
    if isinstance(value, (list, tuple)):
        return np.asarray(value, dtype=float)
    if isinstance(value, str):
        parsed = ast.literal_eval(value)
        return np.asarray(parsed, dtype=float)
    return np.asarray([value], dtype=float)

print(f'train_data shape: {train_data.shape}')
print(f'signal_cols: {len(signal_cols)}')
print(f'hand_names: {len(hand_names)}')

## 4) Distal-finger 3D trajectory plot (moved from hybrid notebook)

In [ ]:
# Create a 3D plot for distal ends of fingers for each hand with color gradient over time

# Filter for distal finger columns
distal_finger_names = [col for col in hand_names if 'distal' in col.lower() and 'finger' in col.lower()]

fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(111, projection='3d')

# Get the first row of data
first_row = train_data.iloc[0]

# Separate by hand (left vs right)
left_fingers = [col for col in distal_finger_names if 'left' in col.lower()]
right_fingers = [col for col in distal_finger_names if 'right' in col.lower()]

# Plot left hand distal fingers with gradient
for col_group in left_fingers:
    base_name = col_group.rsplit('_', 1)[0] if len(col_group) > 2 and col_group[-2] == '_' else col_group

    x_col = [c for c in distal_finger_names if c == base_name + '_x']
    y_col = [c for c in distal_finger_names if c == base_name + '_y']
    z_col = [c for c in distal_finger_names if c == base_name + '_z']

    if x_col and y_col and z_col:
        x = to_series_array(first_row[x_col[0]])
        y = to_series_array(first_row[y_col[0]])
        z = to_series_array(first_row[z_col[0]])

        trajectory_3d = np.column_stack([x, y, z])
        n_points = len(trajectory_3d)

        for i in range(max(n_points - 1, 0)):
            alpha = i / max(n_points - 1, 1)
            color = (0.3 + 0.7 * alpha, 0.3 + 0.7 * alpha, 1.0)
            ax.plot(
                trajectory_3d[i:i+2, 0], trajectory_3d[i:i+2, 1], trajectory_3d[i:i+2, 2],
                color=color, linewidth=2
            )

# Plot right hand distal fingers with gradient
for col_group in right_fingers:
    base_name = col_group.rsplit('_', 1)[0] if len(col_group) > 2 and col_group[-2] == '_' else col_group

    x_col = [c for c in distal_finger_names if c == base_name + '_x']
    y_col = [c for c in distal_finger_names if c == base_name + '_y']
    z_col = [c for c in distal_finger_names if c == base_name + '_z']

    if x_col and y_col and z_col:
        x = to_series_array(first_row[x_col[0]])
        y = to_series_array(first_row[y_col[0]])
        z = to_series_array(first_row[z_col[0]])

        trajectory_3d = np.column_stack([x, y, z])
        n_points = len(trajectory_3d)

        for i in range(max(n_points - 1, 0)):
            alpha = i / max(n_points - 1, 1)
            color = (1.0, 0.3 + 0.7 * alpha, 0.3 + 0.7 * alpha)
            ax.plot(
                trajectory_3d[i:i+2, 0], trajectory_3d[i:i+2, 1], trajectory_3d[i:i+2, 2],
                color=color, linewidth=2
            )

ax.set_xlabel('X (meters)')
ax.set_ylabel('Y (meters)')
ax.set_zlabel('Z (meters)')
ax.set_zlim(bottom=0)
ax.set_title('3D Distal Finger Trajectories - First Row Data\n(Color gradient: lighter=earlier, darker=later)')
plt.tight_layout()
plt.show()

## 5) Run and verify
Run cells top-to-bottom. The final cell should render a 3D plot without `NameError` for `train_data`, `hand_names`, or `signal_cols`.